# Best 100-epoch full pipeline (Audio → Video → Fusion)

This notebook prepares a dedicated config and runs three training stages in sequence:
1. Audio (MIL multiclass) for 100 epochs
2. Video window model for 100 epochs
3. Temporal fusion for 100 epochs using the best audio+video checkpoints produced above

In [ ]:
from pathlib import Path
import json
import os
import subprocess
import sys

ROOT = Path.cwd()
if not (ROOT / "configs" / "master_config.json").exists():
    raise RuntimeError(f"Run this notebook from repo root. Current dir: {ROOT}")

print(f"Repo root: {ROOT}")
print(f"Python: {sys.executable}")

In [ ]:
base_cfg_path = ROOT / "configs" / "master_config.json"
run_cfg_path = ROOT / "configs" / "master_config.best_100_pipeline.json"

cfg = json.loads(base_cfg_path.read_text())

# ---- AUDIO: best-known multiclass MIL config, 100 epochs ----
ac = cfg.setdefault("audio", {})
am = ac.setdefault("model", {})
at = ac.setdefault("training", {})
amil = at.setdefault("sequence_mil", {})

am["dropout"] = 0.15
at["task"] = "multiclass"
at["num_epochs"] = 100
at["lr"] = 7e-05
at["weight_decay"] = 3e-05
at["patience"] = 30
at["checkpoint_dir"] = "checkpoints/audio_best_100"
at["metric"] = "macro_f1"

amil["enabled"] = True
amil["batch_size"] = 8
amil["topk_ratio_pos"] = 0.12
amil["topk_ratio_neg"] = 0.2
amil["eval_pool_ratio"] = 0.12
amil["auto_threshold"] = True
amil["good_window_weight"] = 0.0
amil["use_class_weights"] = True
amil["class_weight_power"] = 0.65
amil["multiclass_eval_mode"] = "topk_per_class"
amil["use_balanced_sampler"] = True
amil["balanced_sampler_power"] = 0.35

# ---- VIDEO: best-known window config, 100 epochs ----
vw = cfg.setdefault("video_window", {})
vwm = vw.setdefault("model", {})
vwt = vw.setdefault("training", {})

vwm["dropout"] = 0.12
vwt["epochs"] = 100
vwt["lr"] = 1e-4
vwt["weight_decay"] = 7e-5
vwt["class_weight_power"] = 1.0
vwt["use_balanced_sampler"] = True
vwt["balanced_sampler_power"] = 0.3
vwt["patience"] = 30
vwt["checkpoint_dir"] = "checkpoints/video_best_100"
vwt["checkpoint_path"] = "checkpoints/video_best_100/window_classifier.pth"

# ---- FUSION: temporal config, 100 epochs ----
fc = cfg.setdefault("fusion", {})
fm = fc.setdefault("model", {})
ft = fc.setdefault("training", {})

fm["arch"] = "temporal"
fm["audio_dim"] = 128
fm["video_dim"] = 128
fm["hidden_dim"] = 192
fm["dropout"] = 0.2
fm["sequence_len"] = 12
fm["temporal_layers"] = 1

ft["num_epochs"] = 100
ft["lr"] = 1e-4
ft["weight_decay"] = 1e-4
ft["batch_size"] = 64
ft["patience"] = 30
ft["checkpoint_dir"] = "checkpoints/fusion_best_100"

run_cfg_path.write_text(json.dumps(cfg, indent=2))
print(f"Wrote config: {run_cfg_path}")

In [ ]:
def run_cmd(cmd):
    print('\n' + '='*90)
    print('RUN:', ' '.join(cmd))
    print('='*90)
    subprocess.run(cmd, check=True, cwd=ROOT)

audio_ckpt = ROOT / "checkpoints" / "audio_best_100" / "best_model.pt"
video_ckpt = ROOT / "checkpoints" / "video_best_100" / "best_model.pt"
fusion_ckpt = ROOT / "checkpoints" / "fusion_best_100" / "best_model.pt"

print('Expected outputs:')
print(' -', audio_ckpt)
print(' -', video_ckpt)
print(' -', fusion_ckpt)

## 1) Train audio model (100 epochs)

In [ ]:
run_cmd([
    sys.executable, '-u', '-m', 'audio.run_audio',
    '--config', str(run_cfg_path),
])

assert audio_ckpt.exists(), f"Audio checkpoint not found: {audio_ckpt}"
print(f"Audio best checkpoint: {audio_ckpt}")

## 2) Train video model (100 epochs)

In [ ]:
run_cmd([
    sys.executable, '-u', '-m', 'video.run_video',
    '--config', str(run_cfg_path),
])

assert video_ckpt.exists(), f"Video checkpoint not found: {video_ckpt}"
print(f"Video best checkpoint: {video_ckpt}")

## 3) Train fusion model (100 epochs) with best audio+video

In [ ]:
run_cmd([
    sys.executable, '-u', '-m', 'fusion.run_fusion',
    '--config', str(run_cfg_path),
    '--audio_checkpoint', str(audio_ckpt),
    '--video_checkpoint', str(video_ckpt),
])

assert fusion_ckpt.exists(), f"Fusion checkpoint not found: {fusion_ckpt}"
print(f"Fusion best checkpoint: {fusion_ckpt}")

## 4) Optional: evaluate best fusion checkpoint

In [ ]:
run_cmd([
    sys.executable, '-u', '-m', 'fusion.run_fusion',
    '--config', str(run_cfg_path),
    '--audio_checkpoint', str(audio_ckpt),
    '--video_checkpoint', str(video_ckpt),
    '--test_only',
    '--checkpoint', str(fusion_ckpt),
])